# Show how extinction curves work with sliders
<em> Just a hack! Don't trust this for your dust! #AstroHackWeek2017



In [1]:
import numpy as np
import pandas as pd
from bokeh.io import output_file, show, push_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider, Select, TextInput
from bokeh.plotting import Figure, output_file, show, output_notebook
from bokeh.models import Arrow, VeeHead, LabelSet

output_file("extinction_models.html")
output_notebook()


Loading BokehJS ...

In [2]:
# I ran   conda install -c bokeh flexx    first


In [5]:
run getFMext

In [6]:
run getMags_ext

In [7]:
x=np.arange(0.45,5.,.01)
invx=1./x


In [8]:
# make a multi-layer pandas frame... fun, but not useful here

# cols = [np.array(['', '', '', 'R2', 'R2', 'R2', 'R3', 'R3', 'R3', 'R4', 'R4', 'R4', 'R5', 'R5', 'R5']),
#         np.array(['A', 'EHK', 'EJH', 'A', 'EHK', 'EJH', 'A', 'EHK', 'EJH', 'A', 'EHK', 'EJH', 'A', 'EHK', 'EJH'])]

# df = pd.DataFrame(index=x, columns=cols)

# for model in ['f99 tables']:
#     for Rlabel,Rv in [('R2',2.1),('R3',3.1),('R4',4.1)]:
#         df[Rlabel, 'A']=getFMext(x,Rv,model)/Rv
#         m=getMags_ext(x,getFMext(x,Rv,model)/Rv)
#         df[Rlabel, 'EHK']=m['H']-m['Ks']
#         df[Rlabel, 'EJH']=m['J']-m['H']
        

In [9]:
cols=['x','invx']

for modellabel in ['','1']:
    for Rlabel in ['','R2','R3','R4','R5']:
        for y in ['A','EHK','EJH', 'EVR', 'AV']:
            cols.append(modellabel+Rlabel+y)    

df = pd.DataFrame(index=x, columns=cols)
df['x']=x
df['invx']=1./x


for modellabel,model in [('','fmunred'),('1','f99 tables')]:
    for Rlabel,Rv in [('R2',2.1),('R3',3.1),('R4',4.1),('R5',5.1)]:
        df[modellabel+Rlabel+'A']=getFMext(x,Rv,model)/Rv
        m=getMags_ext(x,getFMext(x,Rv,model)/Rv)
        df[modellabel+Rlabel+'EHK']=m['H']-m['Ks']
        df[modellabel+Rlabel+'EJH']=m['J']-m['H']
        df[modellabel+Rlabel+'EVR']=m['V']-m['R']
        df[modellabel+Rlabel+'AV']=m['V']

        
# initialize
df['A']=df['R3A']
df['EJH']=df['R3EJH']
df['EHK']=df['R3EHK']
df['EVR']=df['R3EVR']
df['AV']=df['R3AV']


In [10]:
# Not using the column data source...

# plot1 = Figure(width=300, height=300, x_range=[0,5], y_range=[0,2],
#                x_axis_label='lambda (microns)', y_axis_label='A(lambda)', toolbar_location=None)
# plot1.line(df.index.values, df['R3A'].values, line_width=3, line_color='red')
# 
# show(column(plot1))


In [11]:
# Using the column data source

source=ColumnDataSource(df)




In [12]:
plot1 = Figure(width=300, height=300, x_range=[0.4,5], y_range=[0,2],
               x_axis_label='lambda (microns)', y_axis_label='A(lambda)')
r1 = plot1.line('x', 'A', source=source, line_width=3, line_color='black')

plot2 = Figure(width=300, height=300, x_range=[0,2.5], y_range=[0,2], 
               x_axis_label='1/lambda (1/microns)', y_axis_label='A(lambda)')
r2 = plot2.line('invx', 'A', source=source, line_width=3, line_color='black')

plot3 = Figure(width=300, height=300, x_range=[0,.5], y_range=[0,.5],
               x_axis_label='E(H-Ks)', y_axis_label='E(J-H)',
               title='slope = '+str(df.EJH[0.5]/df.EHK[0.5])[0:4]+', A(V) = '+str(df.AV[0.5]/df.EHK[0.5])[0:4]+' run')
plot3.segment(0, 0, 'EHK', 'EJH', source=source, line_width=3, line_color='red')
plot3.add_layout(Arrow(end=VeeHead(fill_color='red',line_color='red'), line_color="red",
                       x_start=0, y_start=0, x_end='EHK', y_end='EJH', source=source))

plot4 = Figure(width=300, height=300, x_range=[0,1.], y_range=[0,5.],
               x_axis_label='E(V-R)', y_axis_label='A(V)',
               title='slope = '+str(df.AV[0.5]/df.EVR[0.5])[0:4]+', A(V) = '+str(df.AV[0.5]/df.EVR[0.5])[0:4]+' run')
plot4.segment(0, 0, 'EVR', 'AV', source=source, line_width=3, line_color='blue')
plot4.add_layout(Arrow(end=VeeHead(fill_color='blue',line_color='blue'), line_color="blue",
                       x_start=0, y_start=0, x_end='EVR', y_end='AV', source=source))




In [13]:
slider1 = Slider(start=1., end=5., value=1., step=.1, title="A(V)")
slider2 = Slider(start=2.1, end=5.1, value=3.1, step=1., title="R(V)")

model = Select(title='Model', value='fmunred', options=['fmunred'] + ['f99 tables'])


def update(source=source, model=model, slider1=slider1, slider2=slider2, plot3=plot3,
           plot4=plot4, window=None):
    
    data = source.data
    y = data['A']
    ejh = data['EJH']
    ehk = data['EHK']
    evr = data['EVR']
    av = data['AV']
    
    mod = model.value
    
    if mod=='fmunred': model=''
    if mod=='f99 tables': model='1'
        
    if slider2.value==2.1:
        y1 = data[model+'R2A']
        ejh1 = data[model+'R2EJH']
        ehk1 = data[model+'R2EHK']
        evr1 = data[model+'R2EVR']
        av1 = data[model+'R2AV']
    if slider2.value==3.1:
        y1 = data[model+'R3A']
        ejh1 = data[model+'R3EJH']
        ehk1 = data[model+'R3EHK']
        evr1 = data[model+'R3EVR']
        av1 = data[model+'R3AV']
    if slider2.value==4.1:
        y1 = data[model+'R4A']
        ejh1 = data[model+'R4EJH']
        ehk1 = data[model+'R4EHK']
        evr1 = data[model+'R4EVR']
        av1 = data[model+'R4AV']
    if slider2.value>4.1:
        y1 = data[model+'R5A']
        ejh1 = data[model+'R5EJH']
        ehk1 = data[model+'R5EHK']
        evr1 = data[model+'R5EVR']
        av1 = data[model+'R5AV']
    for i in range(len(y)):
        y[i] = slider1.value*y1[i]
        ehk[i] = slider1.value*ehk1[i]
        ejh[i] = slider1.value*ejh1[i]
        evr[i] = slider1.value*evr1[i]
        av[i] = slider1.value*av1[i]

    plot3.title.text = title='slope = '+str(ejh[0]/ehk[0])[0:4]+', A(V) = '+str(av[0]/ehk[0])[0:4]+' run'
    plot4.title.text = title='slope = '+str(av[0]/evr[0])[0:4]+', A(V) = '+str(av[0]/evr[0])[0:4]+' run'
    
    source.trigger('change')

    
slider1.js_on_change('value', CustomJS.from_py_func(update))
slider2.js_on_change('value', CustomJS.from_py_func(update))

model.js_on_change('value', CustomJS.from_py_func(update))


t=show(column(row(column(slider1, slider2),model), row(plot1, plot2), row(plot3, plot4)),notebook_handle=True)
